In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'
spark_version = '3.5.7'  # Đảm bảo đúng phiên bản Spark của bạn

packages = f"org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version},org.apache.kafka:kafka-clients:3.5.0"

spark = SparkSession.builder \
    .appName("CNN and Kafka") \
    .config("spark.jars.packages", packages) \
    .config("spark.ui.showConsoleProgress", "false") \
    .master("local[*]") \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d198bdb-4633-4f64-9665-96ff564ef457;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.0 in central
	found com.github.luben#zstd-jni;1.5.5-1 in central
	found org.l

In [2]:
from pyspark.sql.types import ArrayType, FloatType
# Các hằng số từ notebook huấn luyện của bạn
MODEL_PATH = 'cnn_multi_aspect_model.h5'
TOKENIZER_PATH = 'tokenizer.pickle'
MAX_SEQUENCE_LENGTH = 100

ASPECT_COLUMNS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']

# Ánh xạ ngược từ index (0-3) về nhãn gốc (-1, 0, 1, 2)
label_map = {-1: 0, 0: 1, 1: 2, 2: 3}
inverse_label_map = {v: k for k, v in label_map.items()}

# Broadcast đường dẫn file để các executor có thể thấy
sc = spark.sparkContext
broadcasted_model_path = sc.broadcast(MODEL_PATH)
broadcasted_tokenizer_path = sc.broadcast(TOKENIZER_PATH)

# Schema cho đầu ra của UDF: một mảng chứa 8 mảng con (mỗi mảng con 4 xác suất)
schema_output = ArrayType(ArrayType(FloatType()))

In [3]:
from typing import Iterator
from pandas import Series
from pyspark.sql.functions import pandas_udf
@pandas_udf(schema_output)
def predict_sentiments_udf(iterator: Iterator[Series]) -> Iterator[Series]:
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from pickle import load
    from re import sub
    from os import path

    model_path = broadcasted_model_path.value
    tokenizer_path = broadcasted_tokenizer_path.value
    
    if not path.exists(model_path) or not path.exists(tokenizer_path):
        raise FileNotFoundError(f"Model/Tokenizer không tìm thấy trên worker. Đảm bảo {model_path} và {tokenizer_path} có thể truy cập được.")
        
    model = load_model(model_path)
    with open(tokenizer_path, 'rb') as handle:
        tokenizer = load(handle)
    
    def clean_text_udf(text):
        text = str(text).lower()
        text = sub(r'[^\w\s]', '', text)
        text = sub(r'\d+', '', text)
        return text.strip()

    for comments_batch in iterator:
        cleaned_comments = comments_batch.apply(clean_text_udf)
        
        sequences = tokenizer.texts_to_sequences(cleaned_comments)
        padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
        
        if len(padded_sequences) > 0:
            predictions = model.predict(padded_sequences, verbose=0)
            result = [list(map(lambda x: x.tolist(), p)) for p in zip(*predictions)]
        else:
            result = []

        # 5. Trả về batch kết quả
        yield Series(result)

In [4]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
json_fields = [StructField("review_text", StringType())]
for aspect in ASPECT_COLUMNS:
    json_fields.append(StructField(aspect, IntegerType(), True)) # True = nullable
json_schema = StructType(json_fields)

In [5]:
KAFKA_SERVER = "kafka:9092"
TOPIC_NAME = "review_stream"

kafka_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_SERVER) \
    .option("subscribe", TOPIC_NAME) \
    .load()

In [6]:
from kafka import KafkaAdminClient
admin = KafkaAdminClient(bootstrap_servers="kafka:9092")
print(admin.list_topics())

['review_stream']


In [7]:
# 2. Parse JSON và lấy các cột
from pyspark.sql.functions import col, from_json
parsed_df = kafka_df.select(
    col("value").cast("string").alias("json_value")
).select(
    from_json(col("json_value"), json_schema).alias("data")
).select("data.*") 

In [8]:
# 3. Áp dụng Pandas UDF để dự đoán (chỉ cần 'review_text')
predictions_df = parsed_df.withColumn(
    "predictions_prob",
    predict_sentiments_udf(col("review_text"))
)

In [9]:
from pyspark.sql.functions import udf
from numpy import argmax
# 4. Giải nén dự đoán và giữ lại nhãn thật
# Bắt đầu với dataframe chứa nhãn thật và xác suất dự đoán
result_df = predictions_df # predictions_df giờ đã chứa cả cột nhãn thật và predictions_prob

# UDF để map ngược index (0, 1, 2, 3) về nhãn (-1, 0, 1, 2)
udf_inverser = udf(lambda idx: inverse_label_map.get(idx, -99), IntegerType())

for i, aspect in enumerate(ASPECT_COLUMNS):
    # UDF để lấy index có xác suất cao nhất
    udf_extractor = udf(lambda prob_array: int(argmax(prob_array[i])), IntegerType())

    # Lấy ra index dự đoán (0-3)
    result_df = result_df.withColumn(
        f"pred_idx_{aspect}",
        udf_extractor(col("predictions_prob"))
    )
    # Ánh xạ ngược index về nhãn gốc (-1, 0, 1, 2)
    result_df = result_df.withColumn(
        f"pred_{aspect}",
        udf_inverser(col(f"pred_idx_{aspect}"))
    ).drop(f"pred_idx_{aspect}") # Xóa cột index trung gian

# Xóa cột xác suất không cần thiết nữa
result_df = result_df.drop("predictions_prob")
print("Đã định nghĩa luồng xử lý (bao gồm nhãn thật).")

Đã định nghĩa luồng xử lý (bao gồm nhãn thật).


In [10]:
# Chọn các cột cuối cùng để hiển thị
final_output_df = result_df.select(
    "review_text",
    "pred_Quality",
    "pred_Price",
    "pred_Shipping",
    "pred_Shop_Service",
    "pred_Size",
    "pred_Outlook",
    "pred_General",
    "pred_Others"
)

# Chạy stream và hiển thị ra console
query = final_output_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

print("Query đã bắt đầu. Đang chờ dữ liệu từ Kafka...")
print("Chạy cell producer 'sendStream_reviews.ipynb' để gửi dữ liệu.")
print("Nhấn Interrupt Kernel (nút Stop) để dừng stream.")

try:
    query.awaitTermination()
except KeyboardInterrupt:
    print("Đang dừng query...")
    query.stop()
    print("Query đã dừng.")

25/10/25 18:16:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-21d97302-f85d-49c5-9f6d-a6d00a6a0ff5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 18:16:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query đã bắt đầu. Đang chờ dữ liệu từ Kafka...
Chạy cell producer 'sendStream_reviews.ipynb' để gửi dữ liệu.
Nhấn Interrupt Kernel (nút Stop) để dừng stream.
-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+------------+----------+-------------+-----------------+---------+------------+------------+-----------+
|review_text|pred_Quality|pred_Price|pred_Shipping|pred_Shop_Service|pred_Size|pred_Outlook|pred_General|pred_Others|
+-----------+------------+----------+-------------+-----------------+---------+------------+------------+-----------+
+-----------+------------+----------+-------------+-----------------+---------+------------+------------+-----------+



25/10/25 18:17:17 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/10/25 18:17:21 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/10/25 18:17:21 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/10/25 18:17:21 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/10/25 18:17:21 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/10/25 18:17:21 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

StreamingQueryException: [STREAM_FAILED] Query [id = 481095d1-bdc9-43b3-a7f7-6b82e1a22507, runId = 2ed52117-a2e8-4b28-82d6-c60971370d30] terminated with exception: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (f993eba3123d executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_461/1710305793.py", line 18, in predict_sentiments_udf
  File "/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py", line 262, in load_model
    return legacy_sm_saving_lib.load_model(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 72, in error_handler
    del filtered_tb
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/base_layer.py", line 870, in from_config
    raise TypeError(
TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 100], 'dtype': 'float3

In [ ]:
# Chọn các cột cuối cùng để ghi vào memory (bao gồm nhãn thật và dự đoán)
display_columns = ["review_text"]
for aspect in ASPECT_COLUMNS:
    display_columns.append(aspect) # Cột nhãn thật
    display_columns.append(f"pred_{aspect}") # Cột dự đoán

final_output_df_mem = result_df.select(*display_columns) # Dùng *

# Nếu bạn muốn hiển thị kết quả trong một bảng (table) mà bạn có thể query:
query_memory = final_output_df_mem.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("review_predictions_table") \
    .start()

print("Query (memory) đã bắt đầu. Chạy cell tiếp theo để xem kết quả và đánh giá.")

25/10/25 18:19:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5b13dd4b-b5f2-4d88-a070-63e4df887d01. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/25 18:19:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query (memory) đã bắt đầu. Chạy cell tiếp theo để xem kết quả và đánh giá.


25/10/25 18:19:31 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894


In [12]:
from IPython.display import display, clear_output
import time

try:
    while True:
        clear_output(wait=True)
        print("Đang làm mới... (Nhấn Interrupt Kernel để dừng)")
        # Hiển thị bảng từ memory
        display(spark.sql("SELECT * FROM review_predictions_table").toPandas())
        time.sleep(5) # Làm mới sau mỗi 5 giây
except KeyboardInterrupt:
    print("Đã dừng hiển thị.")
    query_memory.stop()

Đang làm mới... (Nhấn Interrupt Kernel để dừng)


,review_text,Price,pred_Price,Shipping,pred_Shipping,Outlook,pred_Outlook,Quality,pred_Quality,Size,pred_Size,Shop_Service,pred_Shop_Service,General,pred_General,Others,pred_Others


Đã dừng hiển thị.


In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
import pandas as pd

print("--- Bắt đầu Đánh giá Mô hình trên Dữ liệu Đã Thu thập từ Stream ---")

# --- 1. Đọc dữ liệu dự đoán và nhãn thật từ memory sink ---
try:
    # Bảng này giờ đã chứa cả nhãn thật (vd: 'Price') và dự đoán (vd: 'pred_Price')
    eval_df = spark.sql("SELECT * FROM review_predictions_table")

    if eval_df.count() == 0:
        print("Chưa có dữ liệu trong bảng 'review_predictions_table'. Hãy đợi stream chạy.")
    else:
        print(f"Đã đọc {eval_df.count()} bản ghi từ memory sink để đánh giá.")

        # --- 2. Đánh giá từng khía cạnh ---
        evaluator_f1 = MulticlassClassificationEvaluator(metricName="f1")
        evaluator_accuracy = MulticlassClassificationEvaluator(metricName="accuracy")

        print("\n--- KẾT QUẢ ĐÁNH GIÁ ---")
        total_accuracy = 0
        total_f1 = 0
        valid_aspects = 0

        for aspect in ASPECT_COLUMNS:
            true_col = aspect
            pred_col = f"pred_{aspect}"

            # Chọn cột nhãn và dự đoán, đổi tên, bỏ null
            aspect_eval_df = eval_df.select(
                col(true_col).cast("double").alias("label"),
                col(pred_col).cast("double").alias("prediction")
            ).na.drop() # Rất quan trọng: Bỏ qua nếu nhãn thật là null (-99 hoặc None)

            count = aspect_eval_df.count()
            if count > 0:
                f1_score = evaluator_f1.evaluate(aspect_eval_df)
                accuracy = evaluator_accuracy.evaluate(aspect_eval_df)
                print(f"Khía cạnh: {aspect} ({count} bản ghi)")
                print(f"  Accuracy: {accuracy:.4f}")
                print(f"  F1-Score: {f1_score:.4f}")
                total_accuracy += accuracy
                total_f1 += f1_score
                valid_aspects += 1
            else:
                 print(f"Khía cạnh: {aspect} - Không có dữ liệu hợp lệ (non-null) để đánh giá.")

        # Tính trung bình nếu có khía cạnh hợp lệ
        if valid_aspects > 0:
            avg_accuracy = total_accuracy / valid_aspects
            avg_f1 = total_f1 / valid_aspects
            print("\n--- Trung bình ---")
            print(f"  Average Accuracy: {avg_accuracy:.4f}")
            print(f"  Average F1-Score: {avg_f1:.4f}")
        print("--------------------")


except Exception as e:
    print(f"Lỗi khi truy vấn hoặc đánh giá bảng 'review_predictions_table': {e}")
    print("Hãy đảm bảo query ghi vào memory sink đang chạy và đã xử lý dữ liệu.")

print("\n--- Đánh giá Hoàn tất ---")

--- Bắt đầu Đánh giá Mô hình trên Dữ liệu Đã Thu thập từ Stream ---
Chưa có dữ liệu trong bảng 'review_predictions_table'. Hãy đợi stream chạy.

--- Đánh giá Hoàn tất ---
